In [ ]:
from predictor import FutPredict 
import pandas as pd

In [10]:
import glob

csv_files = glob.glob("data/*.csv")
df_list = [pd.read_csv(file) for file in csv_files]
merged_df = pd.concat(df_list, ignore_index=True)

merged_df.rename(columns={'FTHG': 'home_team_goal', 'FTAG': 'away_team_goal',
                          'HS': 'home_team_shot', 'AS': 'away_team_shot',
                          'HST': 'home_team_shoton','AST': 'away_team_shoton',
                          'HF': 'home_team_foulcommit', 'AF': 'away_team_foulcommit',
                          'HC': 'home_team_corner', 'AC': 'away_team_corner',
                          'HY': 'home_team_yellowcard', 'AY': 'away_team_yellowcard',
                          'HR': 'home_team_redcard', 'AR': 'away_team_redcard'}, inplace=True)

merged_df_odds = merged_df[['Date', 'HomeTeam', 'AwayTeam', 'home_team_goal', 'away_team_goal',
       'home_team_shot', 'away_team_shot', 'home_team_shoton',
       'away_team_shoton', 'home_team_corner', 'away_team_corner',
       'home_team_foulcommit', 'away_team_foulcommit', 'home_team_yellowcard',
       'away_team_yellowcard', 'home_team_redcard', 'away_team_redcard','B365H','B365D','B365A']]

merged_df["Date"] = pd.to_datetime(merged_df["Date"], format="%d/%m/%Y")
merged_df = merged_df.sort_values(by='Date', ascending=True).reset_index(drop=True)

merged_df_odds["Date"] = pd.to_datetime(merged_df_odds["Date"], format="%d/%m/%Y")
merged_df_odds = merged_df_odds.sort_values(by='Date', ascending=True).reset_index(drop=True)

data = merged_df.copy()
data = data.dropna().reset_index(drop=True)

In [ ]:
predictor = FutPredict(history_size=5)
match_data = predictor.prepare_match_data(data)

cutoff_date = pd.to_datetime('2023-12-06')

historical_features = predictor.calculate_historical_features(
    match_data, cutoff_date)

In [15]:
X, y = predictor.prepare_model_features(historical_features)
model, importance = predictor.train_model(X, y)

In [23]:
future_match_date = pd.to_datetime('2024-01-01')
prediction = predictor.predict_match(
        model=model,
        home_team='Liverpool',
        away_team='Newcastle',
        match_date=future_match_date,
        historical_data=match_data
    )


In [24]:
prediction

,match_date,team,opponent,is_home,win_probability,loss_probability,expected_goals,form_trend,defensive_strength
0,2024-01-01,Liverpool,Newcastle,1,74.605440,25.394560,0.886206,0.042380,-0.061433
1,2024-01-01,Newcastle,Liverpool,0,39.629748,60.370252,0.694639,-0.178104,-0.019233
